In [2]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/red_data.csv')
df

,Unnamed: 0,body,post_title,post_time
0,0,"Insulin, insulin, insulin, insulin.",Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
1,1,The brand name drug my doctor prescribed me is...,Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
2,2,The fact that insulin isn't practically free i...,Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
3,3,You don’t need to ‘produce’ the drugs. You jus...,Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
4,4,"Call me a pessimist, but I'll believe it when ...",Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
...,...,...,...,...
21228,21228,Now try to “change it”,Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27
21229,21229,Why would I try to pass a law prohibiting murd...,Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27
21230,21230,Jesus Christ you are dense. Yes a law prohibit...,Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27
21231,21231,"Damn right I'm being dense, because you are to...",Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27


In [ ]:
df.drop('Unnamed: 0', axis = 1, inplace = True)
df

,body,post_title,post_time
0,"Insulin, insulin, insulin, insulin.",Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
1,The brand name drug my doctor prescribed me is...,Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
2,The fact that insulin isn't practically free i...,Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
3,You don’t need to ‘produce’ the drugs. You jus...,Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
4,"Call me a pessimist, but I'll believe it when ...",Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
...,...,...,...
21228,Now try to “change it”,Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27
21229,Why would I try to pass a law prohibiting murd...,Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27
21230,Jesus Christ you are dense. Yes a law prohibit...,Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27
21231,"Damn right I'm being dense, because you are to...",Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27


In [ ]:
df2 = pd.DataFrame(df)

In [ ]:
from textblob import TextBlob

def get_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity < 0:
        return 'Negative'
    else:
        return 'Neutral'

df2['TextBlob'] = df2['body'].apply(get_sentiment)

print(df2[['TextBlob']])

       TextBlob
0       Neutral
1      Negative
2       Neutral
3      Positive
4       Neutral
...         ...
21228   Neutral
21229  Negative
21230  Positive
21231  Negative
21232   Neutral

[21233 rows x 1 columns]


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

def get_vader_sentiment(text):
    sentiment_scores = sia.polarity_scores(text)
    compound = sentiment_scores['compound']

    if compound > 0:
        return 'Positive'
    elif compound < 0:
        return 'Negative'
    else:
        return 'Neutral'

df2['Vader'] = df2['body'].apply(get_vader_sentiment)

print(df2[['Vader']])

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


          Vader
0       Neutral
1      Positive
2      Negative
3      Positive
4      Negative
...         ...
21228   Neutral
21229  Negative
21230  Negative
21231  Negative
21232  Positive

[21233 rows x 1 columns]


In [ ]:
df2[['TextBlob','Vader']]

,TextBlob,Vader
0,Neutral,Neutral
1,Negative,Positive
2,Neutral,Negative
3,Positive,Positive
4,Neutral,Negative
...,...,...
21228,Neutral,Neutral
21229,Negative,Negative
21230,Positive,Negative
21231,Negative,Negative


In [ ]:
df2

,body,post_title,post_time,TextBlob,Vader
0,"Insulin, insulin, insulin, insulin.",Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57,Neutral,Neutral
1,The brand name drug my doctor prescribed me is...,Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57,Negative,Positive
2,The fact that insulin isn't practically free i...,Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57,Neutral,Negative
3,You don’t need to ‘produce’ the drugs. You jus...,Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57,Positive,Positive
4,"Call me a pessimist, but I'll believe it when ...",Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57,Neutral,Negative
...,...,...,...,...,...
21228,Now try to “change it”,Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27,Neutral,Neutral
21229,Why would I try to pass a law prohibiting murd...,Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27,Negative,Negative
21230,Jesus Christ you are dense. Yes a law prohibit...,Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27,Positive,Negative
21231,"Damn right I'm being dense, because you are to...",Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27,Negative,Negative


# **MODEL begins here!**

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load the pre-trained Multilingual BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Load the pre-trained Multilingual BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=3)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import pandas as pd

# Load your Hinglish dataset
df = pd.read_csv('/content/drive/MyDrive/output.csv')


In [ ]:
df.columns

Index(['Unnamed: 0', 'body', 'post_title', 'post_time', 'score', 'label'], dtype='object')

In [ ]:
# Remove 'Unnamed: 0' and 'label' columns
df = df.drop(['Unnamed: 0', 'label'], axis=1)

In [ ]:
from datasets import Dataset

# Convert pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Split the dataset into train and validation sets
train_dataset = dataset.train_test_split(test_size=0.2)['train']
val_dataset = dataset.train_test_split(test_size=0.2)['test']


In [ ]:
from transformers import BertTokenizer

# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Tokenization function to retain 'score' as 'label'
def tokenize_function(examples):
    tokenized = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)
    tokenized['label'] = examples['score']  # Use 'score' as 'label'
    return tokenized

# Tokenize train and validation datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/957 [00:00<?, ? examples/s]

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

In [ ]:
# Remove 'text' and 'score' columns after tokenization
train_dataset = train_dataset.remove_columns(['text', 'score'])
val_dataset = val_dataset.remove_columns(['text', 'score'])


In [ ]:
# Set the dataset format to PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


In [ ]:
# Print the first entry of the train dataset
print(train_dataset[0])


{'input_ids': tensor([  101, 28335, 10410, 14680,   187, 10376, 10129, 40762, 62137, 25085,
        10148, 16143,   119,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

In [ ]:
from transformers import TrainingArguments

# Set up training arguments with logging and evaluation at every epoch
training_args = TrainingArguments(
    output_dir='./results',                # Output directory for saving model checkpoints
    num_train_epochs=3,                    # Number of training epochs
    per_device_train_batch_size=8,         # Batch size for training
    per_device_eval_batch_size=16,         # Batch size for evaluation
    evaluation_strategy='epoch',           # Evaluate the model at the end of each epoch
    logging_dir='./logs',                  # Directory to store logs
    logging_strategy='epoch',              # Log status after every epoch
    save_strategy='epoch',                 # Save model after every epoch
    load_best_model_at_end=True,           # Load the best model when finished
    metric_for_best_model='accuracy',      # Use accuracy as the evaluation metric
    logging_steps=10,                      # Logging steps (adjust if you want logging during training as well)
)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from transformers import EvalPrediction

In [ ]:
def compute_metrics(pred: EvalPrediction):
    # Extract predictions and labels
    predictions, labels = pred.predictions, pred.label_ids

    # Get the predicted classes (assuming the output is logits)
    predictions = np.argmax(predictions, axis=1)

    # Calculate accuracy
    acc = accuracy_score(labels, predictions)

    # Calculate F1 score (you can adjust average method based on your task)
    f1 = f1_score(labels, predictions, average='weighted')

    return {
        'accuracy': acc,
        'f1_score': f1,
    }


In [ ]:
from transformers import Trainer

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics  # Add a custom metric function if needed (e.g., accuracy, F1 score)
)

# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.951700,0.908166,0.610108,0.552366
2,0.863200,0.787904,0.669675,0.600319
3,0.697600,0.672428,0.723827,0.699031


TrainOutput(global_step=831, training_loss=0.8375067865805506, metrics={'train_runtime': 12486.94, 'train_samples_per_second': 0.531, 'train_steps_per_second': 0.067, 'total_flos': 436505160010752.0, 'train_loss': 0.8375067865805506, 'epoch': 3.0})

In [ ]:
# Example Hinglish text for prediction
example_text = "iphone 14"

# Tokenize the example text
inputs = tokenizer(example_text, return_tensors='pt', padding=True, truncation=True, max_length=128)

# Make predictions
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=-1)

# Convert the prediction back to the label
label_map_reverse = {0: 'negative', 1: 'neutral', 2: 'positive'}
predicted_label = label_map_reverse[predictions.item()]

print(f"Predicted sentiment: {predicted_label}")


Predicted sentiment: neutral


# **Model 2.0**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/red_data_3.csv')

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load the pre-trained Multilingual BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Load the pre-trained Multilingual BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=3)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from datasets import Dataset

# Convert pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Split the dataset into train and validation sets
train_dataset = dataset.train_test_split(test_size=0.2)['train']
val_dataset = dataset.train_test_split(test_size=0.2)['test']


In [ ]:
from transformers import BertTokenizer

# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Tokenization function to retain 'score' as 'label'
def tokenize_function(examples):
    tokenized = tokenizer(examples['body'], padding='max_length', truncation=True, max_length=128)
    tokenized['label'] = examples['score']  # Use 'score' as 'label'
    return tokenized

# Tokenize train and validation datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/957 [00:00<?, ? examples/s]

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

In [ ]:
# Remove 'text' and 'score' columns after tokenization
train_dataset = train_dataset.remove_columns(['body', 'score'])
val_dataset = val_dataset.remove_columns(['body', 'score'])


In [ ]:
# Set the dataset format to PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


In [ ]:
# Print the first entry of the train dataset
print(train_dataset[0])


{'input_ids': tensor([  101, 43958,   169, 12331, 14985, 13262,   119,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

In [ ]:
from transformers import TrainingArguments

# Set up training arguments with logging and evaluation at every epoch
training_args = TrainingArguments(
    output_dir='./results',                # Output directory for saving model checkpoints
    num_train_epochs=3,                    # Number of training epochs
    per_device_train_batch_size=8,         # Batch size for training
    per_device_eval_batch_size=16,         # Batch size for evaluation
    evaluation_strategy='epoch',           # Evaluate the model at the end of each epoch
    logging_dir='./logs',                  # Directory to store logs
    logging_strategy='epoch',              # Log status after every epoch
    save_strategy='epoch',                 # Save model after every epoch
    load_best_model_at_end=True,           # Load the best model when finished
    metric_for_best_model='accuracy',      # Use accuracy as the evaluation metric
    logging_steps=10,                      # Logging steps (adjust if you want logging during training as well)
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics  # Add a custom metric function if needed (e.g., accuracy, F1 score)
)

# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,1.085600,1.073503,0.391667,0.314148
2,0.953700,0.803112,0.554167,0.476434
3,0.733500,0.582669,0.745833,0.735747


TrainOutput(global_step=360, training_loss=0.9242706722683377, metrics={'train_runtime': 4700.9215, 'train_samples_per_second': 0.611, 'train_steps_per_second': 0.077, 'total_flos': 188849655574272.0, 'train_loss': 0.9242706722683377, 'epoch': 3.0})

In [ ]:
# Example Hinglish text for prediction
example_text = "ai"

# Tokenize the example text
inputs = tokenizer(example_text, return_tensors='pt', padding=True, truncation=True, max_length=128)

# Make predictions
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=-1)

# Convert the prediction back to the label
label_map_reverse = {0: 'negative', 1: 'neutral', 2: 'positive'}
predicted_label = label_map_reverse[predictions.item()]

print(f"Predicted sentiment: {predicted_label}")


Predicted sentiment: neutral
